In [1]:
import os
import cv2
import yaml
import random
import shutil
from pathlib import Path
from tensorflow.keras import backend as K
from tensorflow.keras import backend
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd
from keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
import ray
from ray import tune
from tensorflow.keras import callbacks

In [7]:
# Paths
image_folder = "datasets/images/train"
label_folder = "datasets/labels/train"
output_folder = "cropped_images"
yaml_file = 'dataset_config.yaml'

with open(yaml_file, "r") as f:
    data = yaml.safe_load(f)
class_names = data["names"]  # Extract class names list

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Process each label file
for label_file in os.listdir(label_folder):
    if not label_file.endswith(".txt"):
        continue

    image_file = label_file.replace(".txt", ".png")  # Assume PNG
    image_path = os.path.join(image_folder, image_file)

    # Check if PNG exists, otherwise try JPG
    if not os.path.exists(image_path):
        image_file = label_file.replace(".txt", ".jpg")
        image_path = os.path.join(image_folder, image_file)

    if not os.path.exists(image_path):
        print(f"Error: Image {image_file} not found for label {label_file}. Skipping.")
        continue

    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Keep original format
    if image is None:
        print(f"Error: Could not load image {image_path}. Skipping.")
        continue

    h, w = image.shape[:2]

    # If PNG has alpha channel, remove it
    if image.shape[-1] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)

    # Read label file
    with open(os.path.join(label_folder, label_file), "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # Convert YOLO format to pixel coordinates
            x_min = max(0, int((x_center - width / 2) * w))
            y_min = max(0, int((y_center - height / 2) * h))
            x_max = min(w, int((x_center + width / 2) * w))
            y_max = min(h, int((y_center + height / 2) * h))

            # Crop and save
            cropped_img = image[y_min:y_max, x_min:x_max]

            # Check if cropped image is valid
            if cropped_img.size == 0:
                print(f"Warning: Empty crop for {label_file}, skipping.")
                continue

            class_name = class_names[class_id]
            class_folder = os.path.join(output_folder, class_name)
            os.makedirs(class_folder, exist_ok=True)

            # Convert to JPG and save
            save_path = os.path.join(class_folder, f"{label_file.replace('.txt', '')}_{class_id}.jpg")
            cv2.imwrite(save_path, cropped_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

print("Extraction completed.")

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


libpng error: Read Error


Error: Could not load image datasets/images/train/759.png. Skipping.


...

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Extraction completed.


In [9]:
print(len(os.listdir('/Users/johnxie301/.cache/kagglehub/datasets/ronanpickell/b100-lego-detection-dataset/versions/1/cropped_images')))
print(len(os.listdir('/Users/johnxie301/.cache/kagglehub/datasets/ronanpickell/b100-lego-detection-dataset/versions/1/cropped_images/2357')))


200
1438


There are 200 class and each has over 1000 images. To sample test if classification could work well in this case, I will pick only 20 class to work with.

In [7]:
# Set paths
parent_dir = "/Users/johnxie301/.cache/kagglehub/datasets/ronanpickell/b100-lego-detection-dataset/versions/1/cropped_images" 
output_dir = "/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data" 


In [8]:
a = 0
class_20 = []
for dir in os.listdir(parent_dir):
    class_20.append(dir)
    a +=1
    if a >= 20:
        break  
print(class_20)

['3701', '24866', '15379', '18654', '85080', '3069b', '87580', '61409', '15712', '3700', '15573', '48729', '3937', '25269', '11090', '3024', '41740', '32073', '3023', '6558']


In [9]:
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir = os.path.join(output_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


In [10]:
# create sub directories for all 30 classes
for dirs in [train_dir,val_dir,test_dir]:
    for class_name in class_20:
        os.makedirs(os.path.join(dirs,class_name), exist_ok= True)

In [11]:
# move the files to train folder first
for file_name in class_20:
    file_path = os.path.join(parent_dir, file_name)
    for file_nameb in os.listdir(file_path):
        file_togo = os.path.join(file_path,file_nameb)
        if os.path.isfile(file_togo):  
            shutil.move(file_togo, os.path.join(train_dir, file_name,file_nameb))


In [12]:
len(os.listdir(train_dir))

21

In [13]:
# now move 20 percent to test folder

for classes in os.listdir(train_dir):
    file_a = os.path.join(train_dir, classes)
    test_files_q = int(len(os.listdir(file_a)) * 0.2)
    dir_list = [dira for dira in os.listdir(file_a)]
    test_20_pct = random.sample(dir_list,test_files_q)
    for files in test_20_pct:
        test_file = os.path.join(file_a,files)
        if os.path.isfile(test_file):  
            shutil.move(test_file, os.path.join(test_dir, classes,files))

NotADirectoryError: [Errno 20] Not a directory: '/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/.DS_Store'

In [39]:
os.remove('/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/.DS_Store')

In [ ]:
# Same process, but 20% of remaining in train goes to val.

for classes in os.listdir(train_dir):
    file_a = os.path.join(train_dir, classes)
    val_files_q = int(len(os.listdir(file_a)) * 0.2)
    dir_list = [dira for dira in os.listdir(file_a)]
    val_20_pct = random.sample(dir_list,val_files_q)
    for files in val_20_pct:
        val_file = os.path.join(file_a,files)
        if os.path.isfile(val_file):  
            shutil.move(val_file, os.path.join(val_dir, classes,files))

In [15]:
# make this to data frame, then I can apply generators
train_file_path = []
train_category = []

for root,dir,files in os.walk(train_dir):
    for file in files:
        if file.lower().endswith(('.jpg')):
            train_file_path.append(os.path.join(root,file))
            train_category.append(os.path.basename(root))
        else:
            continue
# create a dataframe to check 
df_train = pd.DataFrame({'train_file_path': train_file_path, 'train_category': train_category})
pd.set_option('display.max_colwidth', None)
df_train.head()

,train_file_path,train_category
0,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/3701/1137_110.jpg,3701
1,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/3701/1329_110.jpg,3701
2,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/3701/925_110.jpg,3701
3,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/3701/485_110.jpg,3701
4,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/train/3701/1094_110.jpg,3701


In [16]:
#Same thing but validation
val_file_path = []
val_category = []

for root,dir,files in os.walk(val_dir):
    for file in files:
        if file.lower().endswith(('.jpg')):
            val_file_path.append(os.path.join(root,file))
            val_category.append(os.path.basename(root))
        else:
            continue
df_val = pd.DataFrame({'val_file_path': val_file_path, 'val_category': val_category})
pd.set_option('display.max_colwidth', None)
df_val.head()

,val_file_path,val_category
0,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/val/3701/738_110.jpg,3701
1,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/val/3701/640_110.jpg,3701
2,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/val/3701/1817_110.jpg,3701
3,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/val/3701/36_110.jpg,3701
4,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/val/3701/1241_110.jpg,3701


In [17]:
# and test
test_file_path = []
test_category = []

for root,dir,files in os.walk(test_dir):
    for file in files:
        if file.lower().endswith(('.jpg')):
            test_file_path.append(os.path.join(root,file))
            test_category.append(os.path.basename(root))
        else:
            continue
df_test = pd.DataFrame({'test_file_path': test_file_path, 'test_category': test_category})
pd.set_option('display.max_colwidth', None)
df_test.head()

,test_file_path,test_category
0,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/test/3701/1390_110.jpg,3701
1,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/test/3701/728_110.jpg,3701
2,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/test/3701/650_110.jpg,3701
3,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/test/3701/373_110.jpg,3701
4,/Users/johnxie301/Desktop/Regis_Data_Science/computer_vision_project/lego_data/test/3701/363_110.jpg,3701


In [18]:
# check if its 20 classes in each
print(df_train['train_category'].value_counts())

train_category
6558     921
3023     921
32073    921
41740    921
3024     921
11090    921
24866    472
25269    472
3937     472
48729    472
3701     472
3700     472
15712    472
61409    472
87580    472
3069b    472
85080    472
18654    472
15379    472
15573    472
Name: count, dtype: int64


In [19]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [20]:
res_train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory= None,  
    x_col="train_file_path",
    y_col="train_category",

    target_size=(55, 55), # reshape to 55x55 because all images are around 40 something to 40 something
    batch_size=32, 
    class_mode='categorical', # 20 classes
    shuffle = True) 

res_validataion_generator = train_datagen.flow_from_dataframe(
    dataframe=df_val,
    directory= None,  
    x_col="val_file_path",
    y_col="val_category",
    target_size=(55, 55),
    batch_size=32,
    class_mode='categorical',
    shuffle = False)

res_test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory= None,  
    x_col="test_file_path",
    y_col="test_category",
    target_size=(55, 55),
    batch_size=32,
    class_mode='categorical')

Found 12134 validated image filenames belonging to 20 classes.
Found 6014 validated image filenames belonging to 20 classes.
Found 10612 validated image filenames belonging to 20 classes.


In [29]:
def simple_model(config):
    model = models.Sequential([
        layers.Flatten(input_shape=(55, 55, 3)),  # Assuming RGB images
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(config["num_classes"], activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=config["lr"]),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [33]:
pip install "ray[tune]"



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import argparse
import os
from filelock import FileLock
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.air.integrations.keras import ReportCheckpointCallback
import pandas as pd


def resnet50_train(config):
    try:
        # Create data generators for train, validation, and test data
        res_train_generator = train_datagen.flow_from_dataframe(
            dataframe=df_train,
            directory=None, 
            x_col="train_file_path",
            y_col="train_category",
            target_size=(55, 55),  # Resize images
            batch_size=config["batch_size"],
            class_mode='categorical',
            shuffle=True
        )

        res_validataion_generator = test_datagen.flow_from_dataframe(
            dataframe=df_val,
            directory=None,  
            x_col="val_file_path",
            y_col="val_category",
            target_size=(55, 55),
            batch_size=config["batch_size"],
            class_mode='categorical',
            shuffle=False
        )

        # Create the ResNet50 model
        res_base_model = ResNet50(weights='imagenet', include_top=False, pooling='max')
        x = res_base_model.output
        x = layers.Flatten()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dense(config["num_classes"], activation='softmax')(x)
        
        model = models.Model(inputs=res_base_model.input, outputs=x)

        # Freeze the ResNet50 layers except the last few
        for layer in res_base_model.layers[:-10]:  # Freeze all except last 10 layers
            layer.trainable = False

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Fit the model
        model.fit(
            res_train_generator,
            epochs=config['epochs'],
            validation_data=res_validataion_generator,
            verbose=1,
            callbacks=[ReportCheckpointCallback(metrics={"accuracy": "accuracy"})]
        )
        
        # Report accuracy to Ray Tune
        val_acc = model.evaluate(res_validataion_generator, verbose=0)[1]
        tune.report(accuracy=val_acc)

    except Exception as e:
        print(f"Error occurred during training: {e}")
        tune.report(error=str(e))  # Report the error to Ray Tune

def tune_resnet50():
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(resnet50_train, resources={"cpu": 2, "gpu": 0}),
        tune_config=tune.TuneConfig(
            metric="accuracy",
            mode="max",
            scheduler=sched,
            num_samples=10,
        ),
        run_config=tune.RunConfig(
            name="resnet50_exp",
            stop={"accuracy": 0.9},  # Stop when the desired accuracy is reached, I set it to 0.9
        ),
        param_space={
            "lr": tune.uniform(1e-5, 1e-3),
            "batch_size": tune.choice([16, 32, 64]),
            "epochs": 10,
            "num_classes": 20  
        },
    )
    
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

tune_resnet50()


(resnet50_train pid=9837) Found 12134 validated image filenames belonging to 20 classes.
(resnet50_train pid=9837) Epoch 1/10


(resnet50_train pid=9836) /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
(resnet50_train pid=9836)   self._warn_if_super_not_called()


  1/759 ━━━━━━━━━━━━━━━━━━━━ 39:40 3s/step - accuracy: 0.0000e+00 - loss: 3.5635
  2/759 ━━━━━━━━━━━━━━━━━━━━ 2:58 236ms/step - accuracy: 0.0156 - loss: 3.4791  
  2/380 ━━━━━━━━━━━━━━━━━━━━ 2:38 420ms/step - accuracy: 0.0312 - loss: 3.6292
  3/759 ━━━━━━━━━━━━━━━━━━━━ 2:57 235ms/step - accuracy: 0.0174 - loss: 3.4665
  3/759 ━━━━━━━━━━━━━━━━━━━━ 2:29 198ms/step - accuracy: 0.0069 - loss: 3.2528    
  4/759 ━━━━━━━━━━━━━━━━━━━━ 2:50 226ms/step - accuracy: 0.0247 - loss: 3.4495
(resnet50_train pid=9840) Found 6014 validated image filenames belonging to 20 classes. [repeated 11x across cluster]
(resnet50_train pid=9840) Epoch 1/10 [repeated 5x across cluster]
  1/190 ━━━━━━━━━━━━━━━━━━━━ 11:18 4s/step - accuracy: 0.0312 - loss: 3.3527 [repeated 5x across cluster]
  2/759 ━━━━━━━━━━━━━━━━━━━━ 2:25 192ms/step - accuracy: 0.0000e+00 - loss: 3.2878
  2/190 ━━━━━━━━━━━━━━━━━━━━ 2:24 771ms/step - accuracy: 0.0195 - loss: 3.2991 [repeated 3x across cluster]
 29/759 ━━━━━━━━━━━━━━━━━━━━ 2:24 198

(resnet50_train pid=9836) /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(resnet50_train pid=9836)   _log_deprecation_warning(
(resnet50_train pid=9836) /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(resnet50_train pid=9836)   _log_de

710/759 ━━━━━━━━━━━━━━━━━━━━ 9s 198ms/step - accuracy: 0.1179 - loss: 2.9089
711/759 ━━━━━━━━━━━━━━━━━━━━ 9s 198ms/step - accuracy: 0.1180 - loss: 2.9086
190/190 ━━━━━━━━━━━━━━━━━━━━ 144s 740ms/step - accuracy: 0.0646 - loss: 3.1529 - val_accuracy: 0.0682 - val_loss: 2.9825
(resnet50_train pid=9836) Epoch 2/10
712/759 ━━━━━━━━━━━━━━━━━━━━ 9s 198ms/step - accuracy: 0.1180 - loss: 2.9084
713/759 ━━━━━━━━━━━━━━━━━━━━ 9s 198ms/step - accuracy: 0.1181 - loss: 2.9082
  1/190 ━━━━━━━━━━━━━━━━━━━━ 1:24 446ms/step - accuracy: 0.0938 - loss: 2.8229
714/759 ━━━━━━━━━━━━━━━━━━━━ 8s 198ms/step - accuracy: 0.1181 - loss: 2.9080
715/759 ━━━━━━━━━━━━━━━━━━━━ 8s 198ms/step - accuracy: 0.1182 - loss: 2.9078
  2/190 ━━━━━━━━━━━━━━━━━━━━ 1:49 584ms/step - accuracy: 0.1133 - loss: 2.8365
716/759 ━━━━━━━━━━━━━━━━━━━━ 8s 198ms/step - accuracy: 0.1182 - loss: 2.9076
717/759 ━━━━━━━━━━━━━━━━━━━━ 8s 198ms/step - accuracy: 0.1183 - loss: 2.9074
718/759 ━━━━━━━━━━━━━━━━━━━━ 8s 198ms/step - accuracy: 0.1183 - loss

...

759/759 ━━━━━━━━━━━━━━━━━━━━ 75s 98ms/step - accuracy: 0.4214 - loss: 1.7681 - val_accuracy: 0.4026 - val_loss: 1.8428


2025-03-09 04:22:18,085	ERROR tune_controller.py:1331 -- Trial task failed for trial resnet50_train_fa72d_00008
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/worker.py", line 2771, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=time

(resnet50_train pid=10214) Error occurred during training: report() got an unexpected keyword argument 'accuracy'


2025-03-09 04:22:18,111	ERROR tune.py:1037 -- Trials did not complete: [resnet50_train_fa72d_00000, resnet50_train_fa72d_00001, resnet50_train_fa72d_00002, resnet50_train_fa72d_00003, resnet50_train_fa72d_00004, resnet50_train_fa72d_00005, resnet50_train_fa72d_00006, resnet50_train_fa72d_00007, resnet50_train_fa72d_00008, resnet50_train_fa72d_00009]
2025-03-09 04:22:18,111	INFO tune.py:1041 -- Total run time: 3178.39 seconds (3178.36 seconds for the tuning loop).


Best hyperparameters found were:  {'lr': 0.0009071396946108284, 'batch_size': 64, 'epochs': 10, 'num_classes': 20}
